In [1]:
import sys, os, os.path, glob
import tensorflow as tf

In [2]:
project_folder = %pwd
print("Project folder: ", project_folder)
output_folder = os.path.join(project_folder,'outputs')
data_folder = os.path.join(project_folder, 'data')
train_folder = os.path.join(data_folder, 'train')
validation_folder = os.path.join(data_folder, 'validation')
nb_classes = len(glob.glob(train_folder + "/*"))

Project folder:  D:\fran\Work\eShoConAI\workbench


In [3]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="")
    return graph

In [4]:
# We use our "load_graph" function
graph = load_graph(os.path.join(output_folder,'model.pb'))
#graph = load_graph(os.path.join(output_folder,'tensorflow_inception_graph.pb'))


# We can verify that we can access the list of operations in the graph
#for op in graph.get_operations():
#    print(op.name)

In [5]:
import numpy
from keras.applications.inception_v3 import preprocess_input
labels = ["bracelet","earrings","parasol","thermometer"]

def load_image(img_path):
    from keras.preprocessing import image
    # IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
    IM_WIDTH, IM_HEIGHT = 224, 224 
    img = image.load_img(img_path, target_size=(IM_WIDTH, IM_HEIGHT))
    x = image.img_to_array(img)
    x = numpy.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def predict_inception(graph, image):
    with tf.Session(graph=graph) as sess:
        input_tensor = sess.graph.get_tensor_by_name('input_1:0')
        output = sess.run('output:0',feed_dict ={input_tensor:image})
        return output
        #print (output)

# custom inception
def predict(graph, image):
    with tf.Session(graph=graph) as sess:
        input_tensor = sess.graph.get_tensor_by_name('input_1:0')
        #output_operation = sess.graph.get_operation_by_name('dense_2/Softmax')
        #output_tensor = output_operation.outputs[0]
        #output_tensor = sess.graph.get_tensor_by_name('output:0')
        #t_input = tf.placeholder(numpy.float32, name='our_input')
        output = sess.run('dense_2/Softmax:0',feed_dict ={input_tensor:image})
        return output
        #print (output)

Using TensorFlow backend.


In [ ]:
bracelet_path = os.path.join(validation_folder,'bracelet','3506.jpg')
bracelet_image = load_image(bracelet_path)
preds = predict(graph, bracelet_image)
print (preds)
print (labels[preds.argmax()])

In [ ]:
parasol_path = os.path.join(validation_folder,'parasol','114a.jpg')
parasol_image = load_image(parasol_path)
preds = predict(graph, parasol_image)
print (preds)
print (labels[preds.argmax()])

In [ ]:
with tf.Session(graph=graph) as sess:
    #aa = sess.graph.get_operation_by_name("input")
    aa = sess.graph.get_tensor_by_name("input:0")
    output_tensor = sess.graph.get_tensor_by_name('output:0')
    print (aa)
    print (output_tensor)

In [ ]:
bracelet_image.shape

In [ ]:
preds.argmax()

In [ ]:
from keras import backend as K
K.image_data_format()

In [ ]:
import cntk as C
model = C.load_model(os.path.join(output_folder,'model_cntk.pb'))

In [ ]:
model.arguments[0]

In [ ]:
model.output

In [ ]:
bracelet_path = os.path.join(validation_folder,'bracelet','3506.jpg')
bracelet_image = load_image(bracelet_path)
output = model.eval (bracelet_image)
print (output)
print (labels[output.argmax()])

In [ ]:
parasol_path = os.path.join(validation_folder,'parasol','114a.jpg')
parasol_image = load_image(parasol_path)
output = model.eval (parasol_image)
print (output)
print (labels[output.argmax()])

In [ ]:
def rgb_norm(val):
    #return ((2*(val-1))/255) + 1
    return 2/255*(val-255)+1

In [ ]:
print(rgb_norm(0)) # -1
print(rgb_norm(255)) # 1
print(rgb_norm(187)) # 0.47
print(rgb_norm(123)) #0.04

In [ ]:
1-(-1)